# KNN From Scratch
The point of this is to code a binary classifier KNN from scratch in order to learn the architecture, do some experimenting and feasibility checks. Useful for report material.

Will not be importing any modules and will be enforcing 8 bit integers only. Although I will still have access to things like classes and lists, this will mimic the behaviour of the micro-processor better than otherwise.

## Sorting and counting functions

In [ ]:
#chatgpted argsort which is needed
def mergesort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = mergesort(arr[:mid])
    right = mergesort(arr[mid:])

    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0

    while i < len(left) and j < len(right):
        if left[i] < right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    result.extend(left[i:])
    result.extend(right[j:])
    return result

def argsort(arr):
    indices = list(range(len(arr)))  # Create a list of indices

    # Custom sorting function: Sort indices based on values in arr using mergesort
    def merge_sort_indices(indices):
        if len(indices) <= 1:
            return indices

        mid = len(indices) // 2
        left = merge_sort_indices(indices[:mid])
        right = merge_sort_indices(indices[mid:])

        return merge_indices(left, right)

    def merge_indices(left, right):
        result = []
        i = j = 0

        while i < len(left) and j < len(right):
            if arr[left[i]] < arr[right[j]]:  # Compare values from arr using indices
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1

        result.extend(left[i:])
        result.extend(right[j:])
        return result

    return merge_sort_indices(indices)

def count(values):
    counter_0 = 0
    counter_1 = 0

    for val in values:
        if val == 0:
            counter_0 += 1
        else:
            counter_1 += 1

    if counter_1 > counter_0:
        return 1
    else:
        return 0



In [3]:
def distance(x1,x2):
    total = 0
    for c1,c2 in zip(x1,x2):
        total += (c2-c1)**2
    return total

class KNN:#this will only do binary classification
    def __init__(self, K):#might add weighting later but just uniform for the time being
        """Would recommend K is odd only based on how the counting in prediction is done."""
        self.K = K

    def fit(self,x,y):
        self.x = x
        self.y = y

    def predict(self, input_point):
        #going to calculate distance to everypoint and then sort, will upgrade this list later
        distances = [distance(x, input_point) for x in self.x]
        points = argsort(distances)[:self.K]
        return count(points)

        

        
        



        

        
        

        
